In [1]:
from archappl.client import ArchiverMgmtClient, ArchiverDataClient
from archappl.contrib import get_dataset1

In [2]:
import pandas as pd
def normalize(data, tz='UTC'):
    
    meta = data[0]['meta']  
    payloads = data[0]['data']
    
    payload0 = payloads[0]
    other_val_keys = [k for k in payload0 if k not in ('secs', 'nanos')]
    ts_list = []
    val_list = []
    other_val_dict = dict()
    
    for d in payloads:
        ts_list.append(d['secs'] + d['nanos'] / 1.0e9)
        for k in other_val_keys:
            other_val_dict.setdefault(k, []).append(d[k])
    
    df = pd.DataFrame()
    df['timestamp'] = ts_list
    df.set_index('timestamp', inplace=True)
    for k in other_val_keys:
        df[k] = other_val_dict[k]
    
    idx_utc = pd.to_datetime(df.index, unit='s').tz_localize('UTC')
    if tz != 'UTC':
        df.index = idx_utc.tz_convert('US/Eastern')
    else:
        df.index = idx_utc
        
    return meta, df

def df2h5store(h5filename, dataset, store_key, **kws):
    store = pd.HDFStore(h5filename)
    store.append(store_key, dataset, data_columns=dataset.keys())
    store.get_storer(store_key).attrs.metadata = kws
    store.close()

In [2]:
data_client = ArchiverDataClient()

In [3]:
data_client

[Data Client] Archiver Appliance on: http://127.0.0.1:17665/retrieval/data/getData.json

In [8]:
pv = 'VA:LS1_CA01:BPM_D1129:X_RD'
ts_from = "2020-11-11T13:00:00.000000-05:00" 
ts_to = "2020-11-11T13:00:10.000000-05:00"

In [9]:
data = data_client.get_data(pv, ts_from=ts_from, ts_to=ts_to)

http://127.0.0.1:17665/retrieval/data/getData.json?pv=VA:LS1_CA01:BPM_D1129:X_RD&from=2020-11-11T13:00:00.000000-05:00&to=2020-11-11T13:00:10.000000-05:00


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [67]:
data.shape

(10, 3)

In [72]:
import pytz
pytz.all_timezones;

In [73]:
meta, df = normalize(data, tz='US/Eastern')

In [74]:
store_key = meta.pop('name')
df2h5store('data.h5', df, store_key, **meta)

/home/tong/.local/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'root:calc1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [58]:
df.to_hdf('data_1.h5', 'data', complevel=9)

In [55]:
df.to_hdf?

In [94]:
store = pd.HDFStore('data.h5')

In [95]:
store.keys()

['/root:calc1']

In [96]:
data = store['root:calc1']

In [97]:
data

,val,severity,status
timestamp,,,
2020-11-11 10:59:59.914310217-05:00,0.0,2,5
2020-11-11 11:00:00.913961411-05:00,1.0,2,5
2020-11-11 11:00:01.914007902-05:00,2.0,2,5
2020-11-11 11:00:02.913985729-05:00,3.0,1,6
2020-11-11 11:00:03.913824320-05:00,4.0,1,6
...,...,...,...
2020-11-11 11:59:55.913920403-05:00,6.0,1,4
2020-11-11 11:59:56.914015532-05:00,7.0,1,4
2020-11-11 11:59:57.913804054-05:00,8.0,2,3


In [98]:
ds = store.get_storer('root:calc1')

In [99]:
ds.levels

1

In [104]:
store.close()

"<class 'pandas.io.pytables.HDFStore'>\nFile path: data.h5\nFile is CLOSED"

# test google protobuf

In [26]:
client = ArchiverDataClient()

In [27]:
client.format = 'raw'

In [28]:
pv = 'VA:LS1_CA01:BPM_D1129:X_RD'
ts_from = "2020-11-11T16:00:00.000000-05:00" 
ts_to = "2020-11-11T16:00:10.000000-05:00"

In [62]:
data_raw = client.get_data(pv, ts_from=ts_from, ts_to=ts_to)

In [63]:
data_raw

'\x08\x06\x12\x19VA:FS1_BMS:BPM_D2702:Y_RD\x18�\x0f \x01z\x08\x1b\x02\x03EGU\x12\x01mz\t\x1b\x02\x04PREC\x12\x015\n\n\x08\x06\x12\x19VA:FS1_BMS:BPM_D2702:Y_RD\x18�\x0f \x01z\x08\x1b\x02\x03EGU\x12\x01mz\t\x1b\x02\x04PREC\x12\x015\n\x08�߁\x1b\x03\x10�⟮\x01\x19�6�Q.�J?\n\x08�߁\x1b\x03\x10����\x01\x19�is\x9dvJ?\n\x08�߁\x1b\x03\x10����\x01\x19�"�ȓ�K?\n\x08�߁\x1b\x03\x10�נ�\x02\x19:��5�J?\n\x08�߁\x1b\x03\x10����\x02\x19�\x1b\x01;�\x0c�J?\n\x08�߁\x1b\x03\x10����\x02\x19\x01����J?\n\x08�߁\x1b\x03\x10��ē\x03\x19\x084d\x1b\x01�EK?\n\x08�߁\x1b\x03\x10��Ŭ\x03\x19ɈPv\x1f�J?\n\x08�߁\x1b\x03\x10����\x03\x19q\x05�4��J?\n\x08�߁\x1b\x03\x10��\x03\x19J\x12�\u0602�J?\n'

In [31]:
with open('rawdata.pb', 'w') as fp:
    fp.write(data_raw)

# test getDataAtTime

In [32]:
from phantasy import MachinePortal

In [33]:
mp = MachinePortal("FRIB_VA", "LS1FS1")

[11:44:54.339] INFO: phantasy.library.operation.core: Load new machine: 'FRIB_VA', segment: 'LS1FS1'
[11:44:54.343] INFO: phantasy.library.parser.config: Importing FRIB_VA from /home/tong/Dropbox/phantasy-project/phantasy-machines/FRIB_VA
[11:44:54.347] INFO: phantasy.library.parser.config: Loading machine configuration from /home/tong/Dropbox/phantasy-project/phantasy-machines/FRIB_VA/phantasy.ini
[11:44:54.349] INFO: phantasy.library.operation.lattice: Loading segment: 'LS1FS1'
[11:44:54.354] WARNING: phantasy.library.operation.lattice: Default UNICORN policy will be applied.
[11:44:54.356] WARNING: phantasy.library.operation.lattice: Default device polarity will be applied.
[11:44:54.357] INFO: phantasy.library.operation.lattice: Loading PV data from CSV/SQLite: /home/tong/Dropbox/phantasy-project/phantasy-machines/FRIB_VA/ls1fs1/channels.csv
[11:44:54.359] INFO: phantasy.library.pv.datasource: Set source to be CSV table: channels.csv.
[11:44:54.592] INFO: phantasy.library.operation

In [34]:
bpms = mp.get_elements(type='BPM')

In [35]:
client1 = ArchiverDataClient()

In [36]:
ts_from = "2020-11-11T16:00:00.000000-05:00"
ts_to = "2020-11-11T16:00:10.000000-05:00"

In [37]:
pv = bpms[0].pv(field='X')

In [38]:
pv

['VA:LS1_CA01:BPM_D1129:X_RD']

In [70]:
y_list = []
for i in bpms:
    pv = i.pv('Y')[0]
    r = client1.get_data_at_time([pv], ts_from)
    y_list.append(r[pv]['val'])

In [71]:
y_list

[-7.5525289839741225e-06,
 -0.0003378692784870487,
 -0.00022714350451366475,
 -0.00035938620466179874,
 -0.00021506070816518133,
 -0.00017033183203841838,
 -8.206761737929722e-05,
 2.706095945453e-05,
 4.0625053024427424e-05,
 -0.00023657287097184748,
 -0.0013715791528024176,
 -0.0013907268647780611,
 -0.0017819579784486333,
 -0.0030640266706481265,
 -0.0036135892213284416,
 -0.0030809320146077727,
 -0.002612430718194885,
 -0.0029737587134514303,
 -0.003336662471590865,
 -0.0023560414945679726,
 -0.0011135106499182246,
 -9.779094045353018e-05,
 0.00023328015676632831,
 0.00020554141205939057,
 0.0005121598718576249,
 0.0009863114920885877,
 0.0013595246587858028,
 3.979659868651889e-05,
 -0.00022459243339360225,
 -0.00030940936841608715,
 -0.00020263946285315398,
 -0.000253106048241439,
 -0.000613910451838594,
 -0.0008106892523736271,
 -0.0011041827108744515,
 -0.0004901587387850582,
 1.7201306234108374e-05,
 0.00021000183737732335,
 -7.656593127340524e-05,
 -0.0002894485418267583,
 8.

In [72]:
len(bpms)

75

In [107]:
from archappl.data import iso_to_epoch, epoch_to_iso
import numpy as np

ts0_iso = "2020-11-13T14:10:00.000000-05:00"
ts0_epoch, tz = iso_to_epoch(ts0_iso)
delt_sec = 10
t0 = ts0_epoch - delt_sec
t1 = ts0_epoch + delt_sec
ts_begin = epoch_to_iso(t0)
ts_end = epoch_to_iso(t1)
n_ts = 100
ts_list = [epoch_to_iso(i, tz) for i in np.linspace(t0, t1, n_ts)]

In [108]:
len(ts_list)

100

In [109]:
m = []
pv_list = [i.pv(field='X')[0] for i in bpms]
for i, its in enumerate(ts_list):
    r = client1.get_data_at_time(pv_list, ts=its)
    if r is None:
        print(its)
        continue
    m.append([r[pv]['val'] for pv in pv_list if pv in r])

2020-11-13T14:09:50-05:00
2020-11-13T14:10:10-05:00


In [99]:
r = client1.get_data_at_time(pv_list, ts=its)

In [110]:
r

In [101]:
its

'2020-11-13T11:59:50-05:00'

In [102]:
ts0_iso

'2020-11-13T12:00:00.000000-05:00'

In [111]:
m

[[5.319579096335015e-06,
  -0.0001182167942175107,
  -0.0001693970520155204,
  -2.5985943230920333e-06,
  0.000203876680744769,
  0.00013840821560528741,
  0.00022039418716306028,
  -3.0236934123944374e-05,
  -5.202813335385489e-05,
  -9.958994945870487e-05,
  -0.0007561224485192113,
  -0.0006719292745390743,
  -0.0006364235175502759,
  7.530517682815319e-05,
  -0.0007991099175243109,
  -0.00045679979613291705,
  6.0308612244803e-05,
  0.0016953434224864508,
  0.0016393767779545431,
  0.0011034447370720693,
  0.0008603975058068216,
  0.0016652308245475532,
  0.002021256182567983,
  0.0010345669193767928,
  -6.218785660614452e-05,
  -0.0008330693838556611,
  -0.000771538955911585,
  -9.264376081110139e-05,
  -0.00023896390224078083,
  -0.0002188352095879554,
  -0.0003147175770577362,
  0.0002647810441172307,
  0.0003633983122021671,
  0.0005943304267401555,
  4.611398317228105e-05,
  -0.0002297716991988869,
  4.470014842131804e-05,
  0.0004070520268774235,
  0.0007895749851211675,
  0.0